## LG Aimers 본선 오프라인 해커톤 코드

- 팀명:김승희
- 팀원:김승희

- microsoft/deberta-v3-xsmall Pre-trained 모델을 사용하였습니다. 사전 학습 모델 링크: https://huggingface.co/microsoft/deberta-v3-xsmall  

004 
009 
010 
012
013

015 
026
028 
029 
032 

037 
039 
040 
041 
042 

043 
051 
052
053 
056 

064
066 
068 
071 
072 
073 
078 


### install Library

In [ ]:
!pip install pandas==2.2.1 
!pip install sentence_transformers==2.5.1 
!pip install transformers==4.37.1 
!pip install tqdm==4.65.0 
!pip install pyarrow==15.0.0 
!pip install wandb==0.16.3 
!pip install spacy==3.7.4 
!pip install matplotlib==3.8.3
!pip install bitsandbytes==0.41.1 
!pip install accelerate==0.21.0 
!pip install appdirs==1.4.4 
!pip install datasets==2.18.0 
!pip install fire==0.5.0 
!pip install sentencepiece==0.2.0 
!pip install scipy==1.12.0 
!pip install numpy==1.24.3 
!pip install scikit-learn

# 004

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=5 # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_004/'
    test_preds_number='test_preds_004.npy'
    final_submission_name="submission_004.csv"

    
np.random.seed(CFG.seed)

### Utility Function

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

## Data PreProcess

### Data Load

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train

In [ ]:
df_train['new_text']

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

### max length Check

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

# Train

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

# Inference

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()

# 009

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7 # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_009/'
    test_preds_number='test_preds_009.npy'
    final_submission_name="submission_009.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

# 010

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=9 # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_010/'
    test_preds_number='test_preds_010.npy'
    final_submission_name="submission_010.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 012

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7 # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_012/'
    test_preds_number='test_preds_012.npy'
    final_submission_name="submission_012.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix} is {'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix} is {'None' if pd.isnull(x) else x}.")

    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 013

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=9 # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_013/'
    test_preds_number='test_preds_013.npy'
    final_submission_name="submission_013.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix} is {'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix} is {'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 015

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_015/'
    test_preds_number='test_preds_015.npy'
    final_submission_name="submission_015.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

# True Data 증강

In [ ]:
def augment_data_with_is_format(df):
    # is_converted가 True인 행들만 선택
    df_true = df[df['is_converted'] == True].copy()

    # 'new_text' 열에서 ':'을 ' is'로 변경
    df_true['new_text'] = df_true['new_text'].str.replace(':', ' is ')

    # 원본 데이터프레임에 수정된 행들을 추가
    df_augmented = pd.concat([df, df_true], ignore_index=True)
    
    return df_augmented


df_train_text = augment_data_with_is_format(df_train_text)

In [ ]:
print("증강 이후 shape", df_train_text.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train_text['is_converted'].value_counts()
print(counts)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()

# 026

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=5  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=2
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_026/'
    test_preds_number='test_preds_026.npy'
    final_submission_name="submission_026.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 028

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=5  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=4
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_028/'
    test_preds_number='test_preds_028.npy'
    final_submission_name="submission_028.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
df_test = tabular_to_new_text_data(df_test)



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 029

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=1
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_029/'
    test_preds_number='test_preds_029.npy'
    final_submission_name="submission_029.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 032

In [ ]:

import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=4
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_032/'
    test_preds_number='test_preds_032.npy'
    final_submission_name="submission_032.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 037

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=5  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_037/'
    test_preds_number='test_preds_037.npy'
    final_submission_name="submission_037.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""
    
    columns = [

        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for col_name, prefix in columns:
        # 결측치가 아닌 경우에만 해당 열의 데이터를 포함시켜 문자열을 생성
        df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{x}." if pd.notnull(x) else "")
    
    return df


    # for index, (col_name, prefix) in enumerate(columns):
    #     if index == 0:  
    #         df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
    #     else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
    #         df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    # return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    
    if fold==CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 039

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=9  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_039/'
    test_preds_number='test_preds_039.npy'
    final_submission_name="submission_039.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""
    
    columns = [

        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for col_name, prefix in columns:
        # 결측치가 아닌 경우에만 해당 열의 데이터를 포함시켜 문자열을 생성
        df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{x}." if pd.notnull(x) else "")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    
    if fold==CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 040

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=11  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_040/'
    test_preds_number='test_preds_040.npy'
    final_submission_name="submission_040.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""
    
    columns = [

        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for col_name, prefix in columns:
        # 결측치가 아닌 경우에만 해당 열의 데이터를 포함시켜 문자열을 생성
        df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{x}." if pd.notnull(x) else "")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    
    if fold==CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 041

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=7  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_041/'
    test_preds_number='test_preds_041.npy'
    final_submission_name="submission_041.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 042

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=77
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_042/'
    test_preds_number='test_preds_042.npy'
    final_submission_name="submission_042.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 043

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=43
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_043/'
    test_preds_number='test_preds_043.npy'
    final_submission_name="submission_043.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    

    df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" prefer_ver_count is {x}." if pd.notnull(x) else "")
    df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" ver_win_rate_mean_upper is {x}." if pd.notnull(x) else "")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 051

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=51
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_051/'
    test_preds_number='test_preds_051.npy'
    final_submission_name="submission_051.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    

    df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" prefer_ver_count is {x}." if pd.notnull(x) else "")
    df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" ver_win_rate_mean_upper is {x}." if pd.notnull(x) else "")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()

# 052

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=52
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_052/'
    test_preds_number='test_preds_052.npy'
    final_submission_name="submission_052.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 053

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=53
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_053/'
    test_preds_number='test_preds_053.npy'
    final_submission_name="submission_053.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    

    df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" prefer_ver_count is {x}." if pd.notnull(x) else "")
    df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" ver_win_rate_mean_upper is {x}." if pd.notnull(x) else "")
    











    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 056

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=5  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=56
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_056/'
    test_preds_number='test_preds_056.npy'
    final_submission_name="submission_056.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 064

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=64
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_064/'
    test_preds_number='test_preds_064.npy'
    final_submission_name="submission_064.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
torch.cuda.is_available()

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""
    
    columns = [

        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for col_name, prefix in columns:
        # 결측치가 아닌 경우에만 해당 열의 데이터를 포함시켜 문자열을 생성
        df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{x}." if pd.notnull(x) else "")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    
    if fold==CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:

# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 066

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=66
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_066/'
    test_preds_number='test_preds_066.npy'
    final_submission_name="submission_066.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{'None' if pd.isnull(x) else x}.")
        else:  # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix}:{'None' if pd.isnull(x) else x}.")

    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

# True Data 증강

In [ ]:
def augment_data_with_is_format(df):
    # is_converted가 True인 행들만 선택
    df_true = df[df['is_converted'] == True].copy()

    # 'new_text' 열에서 ':'을 ' is'로 변경
    df_true['new_text'] = df_true['new_text'].str.replace(':', ' is ')

    # 원본 데이터프레임에 수정된 행들을 추가
    df_augmented = pd.concat([df, df_true], ignore_index=True)
    
    return df_augmented


df_train_text = augment_data_with_is_format(df_train_text)

In [ ]:
print("증강 이후 shape", df_train_text.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train_text['is_converted'].value_counts()
print(counts)

In [ ]:
# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # trainer 정의
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='binary')},
        )
        
        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 068

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=68
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_068/'
    test_preds_number='test_preds_068.npy'
    final_submission_name="submission_068.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    

    df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" prefer_ver_count is {x}." if pd.notnull(x) else "")
    df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" ver_win_rate_mean_upper is {x}." if pd.notnull(x) else "")
    











    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()

# 071

In [ ]:
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=71
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_071/'
    test_preds_number='test_preds_071.npy'
    final_submission_name="submission_071.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for index, (col_name, prefix) in enumerate(columns):
        if index == 0:  
            df['new_text'] += df[col_name].apply(lambda x: f"{prefix} is {'None' if pd.isnull(x) else x}.")
        else:   # 첫 번째 요소가 아닐 때는 앞에 띄어쓰기 추가
            df['new_text'] += df[col_name].apply(lambda x: f" {prefix} is {'None' if pd.isnull(x) else x}.")

    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 072

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=72
    FOLD=0
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_072/'
    test_preds_number='test_preds_072.npy'
    final_submission_name="submission_072.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)


### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수 -> 간단 Version
def tabular_to_new_text_data(df):
    
    df['new_text'] = ""

    # 각 컬럼에 대하여 'new_text'에 정보를 추가
    # 결측치인 경우 "None"을 문자열로 추가
    
    # 추가된 컬럼
# Index([
    # 
    # 'expected_budget', 
#          'lead_date', 
#        'customer_history', 
# #lead_from_channel', 
# 'lead_description',

# 'event_name', 
#'prefer_ver_count',


# 'transfer_agreement',

#'ver_win_rate_mean_upper',

#       
    
    
    
    columns = [
    # 'expected_budget', -> 돈 어느정도인지. 단위 환산 하면 좋을듯
#          'lead_date', -> 날짜
#        'customer_history', -> new인지 existing인지 text
# #lead_from_channel', -> text
# 'lead_description', ->
# 'event_name', 
#'prefer_ver_count',
# 'transfer_agreement', Y or N
#'ver_win_rate_mean_upper',
        
        ('lead_description','lead description'),
        ('expected_budget', "expected budget"),
        ('lead_date', "lead date"),
        ('customer_history', "customer history"),
        ('lead_from_channel', "lead from channel"),
        ('event_name', 'event name'),
        ('prefer_ver_count', "prefer ver count"),
        ('transfer_agreement', 'transfer agreement'),
        
        ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        ('bant_submit', "bant submit"),
        ('customer_country', "customer country"),
        ('business_unit', "business unit"),
        ('com_reg_ver_win_rate', "com_reg_ver_win_rate"),
        ('customer_idx', "customer idx"),
        ('customer_type', "customer type"),
        ('enterprise', "enterprise"),
        ('historical_existing_cnt', "historical_existing_cnt"),
        ('id_strategic_ver', "id_strategic_ver"),
        ('it_strategic_ver', "it_strategic_ver"),
        ('idit_strategic_ver', "idit_strategic_ver"),
        ('customer_job', "customer job"),
        ('lead_desc_length', "lead_desc_length"),
        ('inquiry_type', "inquiry type"),
        ('product_category', "product category"),
        ('product_subcategory', "product subcategory"),
        ('product_modelname', "product model name"),
        ('customer_country.1', "customer country.1"),
        ('customer_position', "customer position"),
        ('response_corporate', "response corporate"),
        ('expected_timeline', "expected timeline"),
        ('ver_cus', "ver_cus"),
        ('ver_pro', "ver_pro"),
        ('ver_win_rate_x', "ver_win_rate_x"),
        ('ver_win_ratio_per_bu', "ver_win_ratio_per_bu"),
        ('business_area', "business area"),
        ('business_subarea', "business_subarea"),
        ('lead_owner', "lead_owner"),
        

    ]

    for col_name, prefix in columns:
        # 결측치가 아닌 경우에만 해당 열의 데이터를 포함시켜 문자열을 생성
        df['new_text'] += df[col_name].apply(lambda x: f"{prefix}:{x}." if pd.notnull(x) else "")
    
    return df



In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)

df_train_text

In [ ]:
df_train_text.to_csv('./text_and_label.csv')

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)

# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']




# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    print(f"Inferencing fold {fold}")
    model_path = f"{CFG.output_dir}fold_{fold}_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
    trainer = Trainer(model=model)

    # Test dataset에 대한 예측 수행
    raw_pred, _, _ = trainer.predict(test_ds)
    softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
    
    
    if fold==0 and CFG.train_only_one_fold:
        test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
    else:
        test_preds += softmax_pred / CFG.n_splits 

    # Fold 하나만 학습하는 경우에는 break
    if fold==0 and CFG.train_only_one_fold:
        break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()



# 073

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=73
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_073/'
    test_preds_number='test_preds_073.npy'
    final_submission_name="submission_073.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    









    # 1. bant_submit
    bant_submit_explain = """ Bant submit is The ratio of completed values among the four MQL(Marketing Qualified Lead) components: [1] Budget, [2] Title (customer's position/rank), [3] Needs, and [4] Timeline (desired delivery date)."""

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This customer's bant submit is 1.00, indicates that all components (Budget, Title, Needs, Timeline) are fully completed, meaning all required information is provided, fulfilling the MQL criteria perfectly.",
        0.75: bant_submit_explain + " This customer's bant submit is 0.75, indicates that three out of the four components are fully completed. It means that one piece of information is missing or only partially provided.",
        0.50: bant_submit_explain + " This customer's bant submit is 0.50, indicates that two out of the four components are fully completed. This signifies that only half of the information is fully provided.",
        0.25: bant_submit_explain + " This customer's bant submit is 0.25, indicates that one out of the four components is fully completed. It suggests that most of the information is missing or not sufficiently provided.",
        0.00: bant_submit_explain + " This customer's bant submit is 0.00, indicates that none of the components are completed, meaning no information is provided for the MQL criteria."
    }
    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Customer's country is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The business unit corresponding to the product requested in the MQL (Marketing Qualified Lead) is {x}.")

    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" The opportunity ratio calculated based on Vertical Level 1, business unit, and region is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" The id of the customer's company is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The type of customer is {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " Customer's company is a global company." if x == "Enterprise" else " Customer's company is a small, or medium-sized company.")

    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" The number of times previously converted (sales conversion) is {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " 'id_strategic_ver' is 1.0, which means the 'ID' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " 'it_strategic_ver' is 1.0, which means the 'IT' Business Unit considers the specific Vertical Level 1 area strategically important." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " This customer has either 'id_strategic_ver' or 'it_strategic_ver' with a value of 1.0." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer's job is {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The total length of the Lead Description text written by the customer is {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" The type of customer inquiry is {x}.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" The requested product category is {x}.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" The requested product subcategory is {x}.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" The model name of the requested product is {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Regional information based on the name of the company's corporation (continent) is {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" The customer's position in the company is {x}.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" The name of the responsible corporation is {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer's requested processing schedule is {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and has a Customer type of an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " This customer is within a specific Vertical Level 1 (business area) and belongs to a specific Product Category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" The value multiplied by the ratio of the number of Leads based on Vertical to the overall Lead number and the sales conversion success rate against the number of Leads per Vertical is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" The ratio of the number of sales-converted samples to the number of samples per Business Unit in a specific Vertical Level 1 is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" The customer's business area is {x}.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer's detailed business area is {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" The ID of the salesperson in charge is {x}.")
    
    return df


In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# 078

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset

import transformers
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from scipy.special import softmax

In [ ]:
class CFG:
    model_name='microsoft/deberta-v3-xsmall'
    max_length=1024
    lr=2e-5
    num_labels=2
    batch_size=16
    epoch=10  # 10
    FOLD=0
    grad_acc=4
    weight_decay=0.01
    n_splits=5
    seed=78
    save_total_limit_num=1
    metric_name='f1'
    train_only_one_fold=True
    lead_desc_first=True
    DEMO_TEST=False
    
    # PATH
    train_data_dir='./train.csv'
    test_data_dir='./submission.csv'
    output_dir='./fold_model_save_078/'
    test_preds_number='test_preds_078.npy'
    final_submission_name="submission_078.csv"

    
np.random.seed(CFG.seed)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG.seed)

In [ ]:
df_train = pd.read_csv(CFG.train_data_dir)
df_test = pd.read_csv(CFG.test_data_dir)

### Just for DEMO TEST -> train, test 10개씩
if CFG.DEMO_TEST:
    df_train = df_train.iloc[:10]
    df_test = df_test.iloc[:10]



print(df_train.shape)
print(df_test.shape)

In [ ]:
# 학습 데이터 라벨 분포 확인
counts = df_train['is_converted'].value_counts()
print(counts)

In [ ]:
# Tabular Data를 new_text Data로 변환하는 함수
def tabular_to_new_text_data(df):
    
    # 초기 'new_text' 컬럼 설정
    df['new_text'] = "Review the diverse information of a customer to determine whether they are a converted customer or not."

        # ('lead_description','lead description'),
        # ('expected_budget', "expected budget"),
        # ('lead_date', "lead date"),
        # ('customer_history', "customer history"),
        # ('lead_from_channel', "lead from channel"),
        # ('event_name', 'event name'),
        # ('prefer_ver_count', "prefer ver count"),
        # ('transfer_agreement', 'transfer agreement'),
        
        # ('ver_win_rate_mean_upper', 'ver win rate mean upper'),
        
        
        

    # 고객이 작성한 Lead Description -> len()이 5 이하면 그냥 제거
    # df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) else "")
    df['new_text'] += df['lead_description'].apply(lambda x: f"Customer's written Lead Description text: {x}." if pd.notnull(x) and len(x) > 5 else "")

    # 고객이 희망하는 예산 범위
    df['new_text'] += df['expected_budget'].apply(lambda x: f" The customer's desired budget range is {x}." if pd.notnull(x) else "")

    # Lead 정보 생성일
    df['new_text'] += df['lead_date'].apply(lambda x: f" Lead information creation date: {x}." if pd.notnull(x) else "")

    # 이전에 영업 전환 되었던 이력 여부
    df['new_text'] += df['customer_history'].apply(lambda x: f" Previous sales conversion history (New / Existing): {x}." if pd.notnull(x) else "")

    # Lead 정보가 수집된 채널 이름
    df['new_text'] += df['lead_from_channel'].apply(lambda x: f" Lead information was collected from the channel: {x}." if pd.notnull(x) else "")

    # B2B 마케팅 전환 전에 일어난 마케팅 이벤트
    df['new_text'] += df['event_name'].apply(lambda x: f" The marketing event that occurred before the B2B marketing conversion: {x}." if pd.notnull(x) else "")

    # Lead 정보 국외 반출 동의 여부
    df['new_text'] += df['transfer_agreement'].apply(lambda x: f" Consent to the overseas transfer of Lead information: {x}." if pd.notnull(x) else "")
        
        
    
    ### 아래 2개는 안 씀
    # df['new_text'] += df['prefer_ver_count'].apply(lambda x: f" Customer's country is {x}.")
    # df['new_text'] += df['ver_win_rate_mean_upper'].apply(lambda x: f" Customer's country is {x}.")
    





    # 1. bant_submit
    bant_submit_explain = "Bant submission score quantifies the completeness across the four critical MQL factors: [1] Allocation of funds, [2] Designation (customer's level or title), [3] Demands, and [4] Expected delivery timeline."

    bant_submit_descriptions = {
        1.00: bant_submit_explain + " This client's bant submission level reaches 1.00, showing that all parts (Financial Planning, Job Title, Needs, Project Timeline) have been thoroughly addressed, ensuring that all necessary data is supplied, satisfying the MQL criteria entirely.",
        0.75: bant_submit_explain + " This customer's bant score of 0.75 signifies that three out of the four critical elements are fully addressed. This indicates an incomplete or partially filled piece of the required information.",
        0.50: bant_submit_explain + " This client's bant score of 0.50 reveals that half of the critical components, precisely two out of four, are fully filled out, signifying a partial provision of the necessary details.",
        0.25: bant_submit_explain + " This client's bant score of 0.25 reveals that only one of the four key components is thoroughly completed, implying a substantial absence of necessary information.",
        0.00: bant_submit_explain + " A bant submit rate of 0.00 for this customer shows that none of the essential elements have been addressed, resulting in a lack of information for meeting the MQL standards."
    }

    df['new_text'] += df['bant_submit'].map(bant_submit_descriptions)

    # 2. customer_country
    df['new_text'] += df['customer_country'].apply(lambda x: f" Origin country of the customer is {x}.")

    # 3. business_unit
    df['new_text'] += df['business_unit'].apply(lambda x: f" The {x} division is responsible for the product mentioned in the MQL.")
 
    # 4. com_reg_ver_win_rate
    df['new_text'] += df['com_reg_ver_win_rate'].apply(lambda x: f" Calculated success ratio across Vertical Level 1, business unit, and geographical area is {x}.")

    # 5. customer_idx
    df['new_text'] += df['customer_idx'].apply(lambda x: f" Customer's company identification number is {x}.")

    # 6. customer_type
    df['new_text'] += df['customer_type'].apply(lambda x: f" The category of the customer falls into {x}.")

    # 7. enterprise
    df['new_text'] += df['enterprise'].apply(lambda x: " This customer operates a worldwide enterprise." if x == "Enterprise" else " This customer's business is considered small to medium.")
    
    # 8. historical_existing_cnt
    df['new_text'] += df['historical_existing_cnt'].apply(lambda x: f" Historical conversion count stands at {x}.")

    # 9. id_strategic_ver
    df['new_text'] += df['id_strategic_ver'].apply(lambda x: " With 'id_strategic_ver' at 1.0, it's clear that the 'ID' Business Unit values the specific Vertical Level 1 for strategic reasons." if x == 1.0 else "")

    # 10. it_strategic_ver
    df['new_text'] += df['it_strategic_ver'].apply(lambda x: " A 'it_strategic_ver' rating of 1.0 underlines the 'IT' Business Unit's strategic focus on the specific Vertical Level 1." if x == 1.0 else "")

    # 11. idit_strategic_ver
    df['new_text'] += df['idit_strategic_ver'].apply(lambda x: " For this customer, at least one of 'id_strategic_ver' or 'it_strategic_ver' is marked as 1.0, indicating strategic priority." if x == 1.0 else "")

    # 12. customer_job
    df['new_text'] += df['customer_job'].apply(lambda x: f" This customer holds the position of {x}.")

    # 13. lead_desc_length
    df['new_text'] += df['lead_desc_length'].apply(lambda x: f" The comprehensive length of the Lead Description authored by the customer measures {x}.")

    # 14. inquiry_type
    df['new_text'] += df['inquiry_type'].apply(lambda x: f" Customer's query falls under the {x} category.")

    # 15.product_category
    df['new_text'] += df['product_category'].apply(lambda x: f" Customer has requested a product from the {x} category.")

    # 16.product_subcategory
    df['new_text'] += df['product_subcategory'].apply(lambda x: f" Customer has indicated interest in the {x} subcategory.")

    # 17.product_modelname
    df['new_text'] += df['product_modelname'].apply(lambda x: f" Customer requested the product model {x}.")

    # 18. customer_country.1
    df['new_text'] += df['customer_country.1'].apply(lambda x: f" Continental details, based on the corporation's name, reveal {x}.")

    # 19.customer_position
    df['new_text'] += df['customer_position'].apply(lambda x: "" if x == "none" else f" This customer holds the title of {x} in the company.")

    # 20.response_corporate
    df['new_text'] += df['response_corporate'].apply(lambda x: f" Responsibility falls under the corporation {x}.")

    # 21. expected_timeline
    df['new_text'] += df['expected_timeline'].apply(lambda x: f" The customer has specified a processing timeframe of {x}.")

    # 22. ver_cus
    df['new_text'] += df['ver_cus'].apply(lambda x: " This client belongs to a particular Vertical Level 1 sector and is identified as an end-user." if x == 1 else "")

    # 23. ver_pro
    df['new_text'] += df['ver_pro'].apply(lambda x: " Situated in a specific Vertical Level 1, this customer is categorized under a distinct product category." if x == 1 else "")

    # 24. ver_win_rate_x
    df['new_text'] += df['ver_win_rate_x'].apply(lambda x: f" A value resulting from the multiplication of the Vertical Leads ratio to total Leads and their conversion success rate is {x}.")

    # 25. ver_win_ratio_per_bu
    df['new_text'] += df['ver_win_ratio_per_bu'].apply(lambda x: f" In a certain Vertical Level 1, the proportion of successfully converted samples against total business unit samples is {x}.")

    # 26. business_area
    df['new_text'] += df['business_area'].apply(lambda x: f" This customer operates within the {x} sector.")

    # 27. business_subarea
    df['new_text'] += df['business_subarea'].apply(lambda x: f" The customer operates in the detailed sector of {x}.")

    # 28. lead_owner
    df['new_text'] += df['lead_owner'].apply(lambda x: f" Assigned salesperson's identification number is {x}.")
    
    return df

In [ ]:
df_train = tabular_to_new_text_data(df_train)

In [ ]:
df_train_text = df_train[['new_text', 'is_converted']]
df_train_text['is_converted'] = df_train_text['is_converted'].astype(int)
df_train_text.to_csv('./text_and_label.csv')

df_train_text

In [ ]:
# 변형한 train dataset에서 가장 긴 문장 가져오기

changed_train_text = pd.read_csv('./text_and_label.csv')
sample_text_id = changed_train_text['new_text'].apply(len).idxmax()
sample_text = changed_train_text.iloc[sample_text_id]['new_text']

print('sample_text len() : ', len(sample_text))
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

tokenized_text = tokenizer(sample_text, max_length=1024, padding=True, truncation=True)
token_length = len(tokenized_text["input_ids"])

print('sample_text token length:', token_length)

In [ ]:
# 라벨 불균형 weight 넣기
from sklearn.utils.class_weight import compute_class_weight


# 클래스 가중치 계산
y_train = df_train_text['is_converted'].values
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


# 토크나이저, 토큰화 함수
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['new_text'], max_length=CFG.max_length, padding=True, truncation=True)

    tokenized_inputs['labels'] = examples['is_converted']
    return tokenized_inputs
    
# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
fold_scores = []


for fold, (train_index, valid_index) in enumerate(skf.split(df_train_text, df_train_text['is_converted'])):
    print(f'FOLD {fold}')
    
    if fold == CFG.FOLD:
        # 데이터셋 분할
        train_df = df_train_text.iloc[train_index]
        valid_df = df_train_text.iloc[valid_index]
        
        train_ds = Dataset.from_pandas(train_df)
        valid_ds = Dataset.from_pandas(valid_df)
        
        # 데이터셋 인코딩
        train_ds_enc = train_ds.map(tokenize_function, batched=True)
        valid_ds_enc = valid_ds.map(tokenize_function, batched=True)
        
        # 모델 정의
        model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name,
                                                                num_labels=CFG.num_labels)

        
        # TrainingArguments 정의
        # num_steps = len(train_df) // (CFG.batch_size * CFG.grad_acc)
        training_args = TrainingArguments(
            output_dir=f"{CFG.output_dir}fold_{fold}",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=CFG.lr,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.epoch,
            weight_decay=CFG.weight_decay,
            load_best_model_at_end=True,
            metric_for_best_model=CFG.metric_name,
            report_to='none', 
            save_total_limit = CFG.save_total_limit_num,
        )
        
        # CustomTrainer 정의
        class CustomTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights

            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get('logits')
                if self.class_weights is not None:
                    loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(self.args.device))
                    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                else:
                    loss = outputs.loss
                return (loss, outputs) if return_outputs else loss
        
        # CustomTrainer 인스턴스 생성 및 사용
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds_enc,
            eval_dataset=valid_ds_enc,
            tokenizer=tokenizer,
            compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')},
            class_weights=class_weights_tensor
        )

        # 학습과 검증
        trainer.train()
        eval_result = trainer.evaluate()
        
        fold_scores.append(eval_result['eval_f1'])
        print(f"Fold {fold} F1 Score: {eval_result['eval_f1']}")
        
        # 모델 저장
        trainer.save_model(f"{CFG.output_dir}fold_{fold}_model")
        
        # # Fold 하나만 하는 경우 break
        # if fold == 0 and CFG.train_only_one_fold:
        break

# 평균 F1 Score 출력
if fold_scores: 
    print(f"Fold {CFG.FOLD} F1 Score: {fold_scores[0]}")

In [ ]:
# Test Data 역시 Train과 동일하게 Tabular -> Text 작업을 거친다.
df_test = tabular_to_new_text_data(df_test)


# ### Text 붙이기
# # Customer가 남긴 텍스트 열 이름을 'lead_desc'라고 가정 (오프라인 대회 때 변경)
# if CFG.lead_desc_first:
#     df_test['new_text'] = "customer text:" + df_test['lead_desc'] + " " + df_test['new_text']
# else:
#     df_test['new_text'] = df_test['new_text'] + " " + "customer text:" + df_test['lead_desc']



# 토크나이저 정의
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, use_fast=True)

def prepare_test_data(test_texts):
    tokenized_data = tokenizer(test_texts.to_list(), padding=True, truncation=True, max_length=CFG.max_length, return_tensors="pt")
    return tokenized_data


tokenized_test_data = prepare_test_data(df_test['new_text'])
test_ds = Dataset.from_dict(tokenized_test_data)

In [ ]:
test_preds = np.zeros((len(tokenized_test_data["input_ids"]),))

# 각 fold 모델에 대한 예측 수행
for fold in range(CFG.n_splits):
    if fold == CFG.FOLD:
        print(f"Inferencing fold {fold}")
        model_path = f"{CFG.output_dir}fold_{fold}_model"
        model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=CFG.num_labels)
        trainer = Trainer(model=model)

        # Test dataset에 대한 예측 수행
        raw_pred, _, _ = trainer.predict(test_ds)
        softmax_pred = softmax(raw_pred, axis=1)[:, 1]  # 이진 분류인 경우, 클래스 '1'의 확률을 선택
        
        
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            test_preds += softmax_pred # Fold 하나만 하는 경우에는 CFG.n_splits 로 나누지 않는다.
        else:
            test_preds += softmax_pred / CFG.n_splits 

        # Fold 하나만 학습하는 경우에는 break
        if fold==CFG.FOLD and CFG.train_only_one_fold:
            break


print(test_preds.shape)
np.save(CFG.test_preds_number, test_preds)

In [ ]:
# 메모리 관리를 위한 모델 객체 삭제
import gc

del model
torch.cuda.empty_cache()
gc.collect()


# Ensemble

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

In [ ]:

test_preds_004= np.load('./test_preds_004.npy') 
test_preds_009= np.load('./test_preds_009.npy') 
test_preds_010= np.load('./test_preds_010.npy') 
test_preds_012= np.load('./test_preds_012.npy') 
test_preds_013= np.load('./test_preds_013.npy') 
 
test_preds_015= np.load('./test_preds_015.npy') # 0.025에서 0.547
test_preds_026= np.load('./test_preds_026.npy') # 0.05에서 0.538
test_preds_028= np.load('./test_preds_028.npy') # 0.02에서 0.532
test_preds_029= np.load('./test_preds_029.npy') # 0.02에서 0.532
test_preds_032= np.load('./test_preds_032.npy') # 0.04에서 0.530

test_preds_037= np.load('./test_preds_037.npy') # 0.01에서 0.544
test_preds_039= np.load('./test_preds_039.npy') # 0.05에서 0.532
test_preds_040= np.load('./test_preds_040.npy') 
test_preds_041= np.load('./test_preds_041.npy')
test_preds_042= np.load('./test_preds_042.npy') 

test_preds_043= np.load('./test_preds_043.npy') 
test_preds_051= np.load('./test_preds_051.npy') 
test_preds_052= np.load('./test_preds_052.npy') 
test_preds_053= np.load('./test_preds_053.npy') 
test_preds_056= np.load('./test_preds_056.npy') 

test_preds_064= np.load('./test_preds_064.npy') 
test_preds_066= np.load('./test_preds_066.npy') 
test_preds_068= np.load('./test_preds_068.npy') 
test_preds_071= np.load('./test_preds_071.npy') 
test_preds_072= np.load('./test_preds_072.npy') 

test_preds_073= np.load('./test_preds_073.npy') 
test_preds_078= np.load('./test_preds_078.npy') 






In [ ]:


test_preds = (
    
    (
        


    (0.02)*test_preds_004 + 
    (0.02)*test_preds_009 + 
    (0.01)*test_preds_010 +
    (0.02)*test_preds_012 + 
    (0.02)*test_preds_013 + 
    
    
    (0.06)*test_preds_015  +
    (0.03)*test_preds_026  +
    (0.03)*test_preds_028 + 
    (0.02)*test_preds_029 + 
    (0.03)*test_preds_032 +
    
    (0.06)*test_preds_037 +
    (0.03)*test_preds_039 + 
    (0.02)*test_preds_040 +
    (0.03)*test_preds_041 +
    (0.03)*test_preds_042 +
    
    (0.15)*test_preds_043 +
    (0.03)*test_preds_051 +
    (0.01)*test_preds_052 +
    (0.01)*test_preds_053 +
    (0.01)*test_preds_056 +
    
    (0.02)*test_preds_064 +
    (0.16)*test_preds_066 +
    (0.01)*test_preds_068 +
    (0.02)*test_preds_071 +
    (0.07)*test_preds_072 +
    
    
    (0.07)*test_preds_073 +
    (0.01)*test_preds_078 
    
    
    
    
    
    
    
    
    

    )


)



In [ ]:
df_sub = pd.read_csv("./submission.csv")
threshold = 0.163  # 조정할 수 있음
df_sub["is_converted"] = (test_preds > threshold).astype(int)

counts = df_sub['is_converted'].value_counts()
print(counts)

In [ ]:
df_sub.to_csv("submission.csv", index=False)
counts = df_sub['is_converted'].value_counts()
print(counts)

In [ ]:
df_sub